# Demand modeling using discrete choice models

Travel demand models use current travel behavior to predict future travel patterns from a sample of travel behavior data. In this WA and your project you are required to model travel demand using models explored in class and in this WA.

This notebook introduces regression models.  The focus here will be on fitting
these types of models to data, using Python statistical modeling
libraries in the Jupyter notebook environment. 

We begin by importing the libraries that we will be using.

In [1]:
import statsmodels.api as sm
from statsmodels.formula.api import logit
import numpy as np
import pandas as pd
import scipy as scp

In [2]:
#### This week we will use trip file, let's save file location path in a variable url, as usual
urltripv = "tripv2pub.csv"

#### Read the .csv files using `pandas` libriary and store it as a `pandas` Data Frame
dtripv = pd.read_csv(urltripv)

#### Let's now check what data type we stored into `df`. This code will output object type
pd.set_option('display.max_columns', None)

## Discrete Choice Models

**Discrete choice models** are used to explain or predict a choice from a set of two or more discrete (i.e. distinct and separable; mutually exclusive) alternatives.
In this course we will be using this modeling approach to carry out **Binary** (logit and probit) models and **Multinomial Logit Model** (more then two choices). 
All three of them have the same structure:
- [Logit](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.Logit.html)(endog, exog, check_rank=True)
- [Probit](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.Probit.html)(endog, exog, check_rank=True)
- [MNLogit](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.MNLogit.html)(endog, exog, check_rank=True)

where

_endog_ is an 1-d endogenous response variable (the dependent variable), as a _ndarray_.

_exog_ is a $nobs$ * $k$ array where $nobs$ is the number of observations and $k$ is the number of regressors (independent variables). An intercept is not included by default and should be added by the user, also in a _ndarray_ format. 

_check_rank_ (boolean) is a check of _exog_ rank to determine model degrees of freedom. Default is $True$. Setting to $False$ reduces model initialization time when _exog.shape[1]_ is large.

## Binary Logit Model

In [3]:
dtripv.head()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP
0,9000013002,1,1,1,900001300201,1,2,1,1,1,2,75,2,3,1,50,15,1435,1450,1,1,3,2,1,2,2,1,-1,-1,-1,-1,-1,-1,0,2,1,1,2,1,-1.0,-1,-1,15,2,3.902424,1.608362e+06,0.0,1.569852e+06,90000130020101,3.902424,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,39,2,2,1,1,2,1,7,3,2,3,11
1,9000013002,1,2,2,900001300201,1,2,1,1,1,2,-9,2,3,15,1,10,1605,1615,1,1,3,2,1,2,2,1,-1,-1,-1,-1,-1,-1,0,2,1,1,2,1,-1.0,-1,-1,1,1,3.902424,1.608362e+06,0.0,1.569852e+06,90000130020102,3.902424,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,39,2,2,1,1,2,1,7,3,2,3,11
2,9000013002,2,1,1,900001300202,1,-1,1,1,1,2,10,2,2,1,80,30,700,730,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,12,-1,17.077067,2.208973e+06,0.0,2.274239e+06,90000130020201,17.077067,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11
3,9000013002,2,2,2,900001300202,1,-1,1,1,1,2,10,2,5,12,80,10,740,750,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,12,-9,4.748291,2.208973e+06,0.0,2.274239e+06,90000130020202,4.748291,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11
4,9000013002,2,3,3,900001300202,1,-1,1,1,1,2,-9,2,2,12,1,30,800,830,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,-9,14.098819,2.208973e+06,0.0,2.274239e+06,90000130020203,14.098819,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11


In [4]:
dtripv.describe()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP
count,3.107400e+04,31074.000000,31074.000000,31074.000000,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.00000,31074.000000,31074.000000,3.107400e+04,3.107400e+04,3.107400e+04,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000
mean,9.000116e+09,1.681438,2.437826,2.435702,7.619426e+11,1.048529,1.141308,0.775407,1.722276,3.810066,1.988704,95.176611,1.989026,3.156723,7.185943,26.856311,24.547821,1329.818337,1363.067613,1.041063,1.076945,3.906159,1.997265,1.243644,1.129336,0.185267,-0.226427,-0.644236,-0.880543,-0.954464,-0.976830,-0.993338,-0.999710,0.414140,1.583124,3.333526,0.986774,1.423505,3.786606,-0.802894,-0.986677,82.358853,7.43390,-0.210272,13.965150,8.166137e+06,5.815804e+06,2.302368e+06,9.000116e+13,7.527313,397.951262,5.351805,2.058506,1.654921,1.775729,5.143046,2.652346,1.924117,1.993242,53.722565,7.260636,2.677029,2.821941,2.134453,5.812737,2.292109,3.530926,1.653086,4.137478,1.188968,202211.691543,1.303694,1029.494240,46.800830,1.430617,1.126504,0.855892,2.859014,1.916071,1.268198,4.716097,4.710755,1.503926,1.724657,7.415814
std,5.829379e+04,0.999429,1.791504,1.789758,3.243517e+11,0.214885,1.341822,0.787571,0.447884,1.966057,0.105681,164.311604,0.104181,1.445374,6.851472,27.561324,46.483373,422.927312,425.447364,0.514696,1.094555,5.054558,3.477865,0.429287,0.982799,1.336076,1.215187,0.911104,0.563497,0.351382,0.254039,0.137048,0.029476,3.387813,0.891605,14.903745,0.833239,0.975625,15.956259,8.739298,0.207546,909.149169,6.99431,4.073497,85.416700,1.083988e+07,9.561997e+06,5.831630e+06,5.829379e+08,32.176230,68.455342,4.189875,0.761605,0.813206,0.417108,2.485935,1.011683,0.264816,0.794912,24.895858,2.861425,11.855930,1.446787,1.078400,3.251390,0.856474,1.552468,1.166259,1.665612,0.391490,20.136521,0.947416,4.982031,20.765643,0.931312,0.872042,0.646882,12.512009,0.277285,0.443029,2.409187,19.318665,0.499993,1.567172,2.445626
min,9.000013e+09,1.000000,1.000000,1.000000,-1.000000e+00,1.000000,-1.000000,-9.000000,1.000000,1.000000,1.000000,-9.000000,1.000000,-9.000000,-9.000000,1.000000,-9.000000,-9.000000,-9.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,1.000000,-9.000000,-1.000000,-1.000000,-9.000000,-1.000000,-1.000000,-1.000000,1.00000,-9.000000,-9.000000,3.369513e+04,0.000000e+00,0.000000e+00,9.000013e+13,-9.000000,272.700000,1.000000,1.000000,

## Models

1. Explore the factors impacting **public transit** usage using *one* independent variable
2. Explore the factors impacting **public transit** usage using *two* independent variable
3. Explore the factors impacting **public transit** usage using *multiple* independent variable

- Explore use of *Logit* and *Probit* models
- Learn to use methof *from_formula*

In [5]:
dtripv.groupby(['R_RACE']).size()

R_RACE
1     26992
2      1713
3      1428
4       178
5       109
6       116
97      538
dtype: int64

In [6]:
### Creating new variables that will be used in teh models

dtripv["PUBTRANSy"] = 0
dtripv["HHFAMINC_IMP_LOWx"] = 0
dtripv["EDUC_HIx"] = 0
dtripv["R_AGE_YAx"] = False
dtripv["R_SEX_Mx"] = 0
dtripv["PRMACT_RETx"] = False
dtripv["HHFAMINC_IMP_HIx"] = 0
dtripv["R_RACE_WHITEx"] = 0
dtripv["HHSIZE1&2x"] = False
dtripv["URBANx"] = False


dtripv.loc[:, "PUBTRANSy"] = dtripv.loc[:, 'PUBTRANS'].map({1: 1, 2: 0})

dtripv.loc[:, "HHFAMINC_IMP_LOWx"] = dtripv.loc[:, 'HHFAMINC_IMP'].map({1: 1,
                                                                        2: 1,
                                                                        3: 1,
                                                                        4: 1,
                                                                        5: 1,
                                                                        6: 0,
                                                                        7: 0,
                                                                        8: 0,
                                                                        9: 0,
                                                                        10: 0,
                                                                        11: 0})


dtripv.loc[:, "EDUC_HIx"] = dtripv.loc[:, 'EDUC'].map({1: 0,
                                                    2: 0,
                                                    3: 0,
                                                    4: 0,
                                                    5: 0,
                                                    6: 1,
                                                    7: 1,
                                                    8: 1,
                                                    -1: 0})

conditionageYA = ((dtripv['R_AGE'] > 17) & (dtripv['R_AGE'] <27))
dtripv.loc[:, "R_AGE_YAx"] = conditionageYA

dtripv.loc[:, "R_SEX_Mx"] = dtripv.loc[:, 'R_SEX_IMP'].map({1: 1, 2: 0})

dtripv.loc[:, "PRMACT_RETx"] = dtripv.loc[:, 'PRMACT'].map({-1: False, 1: False, 2: False, 3: False, 4: False, 5: True, 97: False})


dtripv.loc[:, "HHFAMINC_IMP_HIx"] = dtripv.loc[:, 'HHFAMINC_IMP'].map({1: 0, 2: 0, 3: 0, 4: 0,
                                                                       5: 0, 6: 0, 7: 0, 8: 1,
                                                                       9: 1, 10: 1, 11: 1})

dtripv.loc[:, "R_RACE_WHITEx"] = dtripv.loc[:, 'R_RACE'].map({1: 1,
                                                        2: 0,
                                                        3: 0,
                                                        4: 0,
                                                        5: 0,
                                                        6: 0,
                                                        97: 0})

dtripv.loc[:, "HHSIZE1&2&3x"] = dtripv['HHSIZE'] < 4

conditionageUR = ((dtripv['URBAN'] < 4))
dtripv.loc[:, "URBANx"] = conditionageUR



In [7]:
df = dtripv.dropna(subset = ['PUBTRANSy', 'HHFAMINC_IMP_LOWx', "EDUC_HIx", "R_AGE_YAx", "R_SEX_Mx", 
                            "PRMACT_RETx", "HHFAMINC_IMP_HIx", "R_RACE_WHITEx", "HHSIZE1&2x", "URBANx"], inplace = False)


In [8]:
df.head()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,PUBTRANSy,HHFAMINC_IMP_LOWx,EDUC_HIx,R_AGE_YAx,R_SEX_Mx,PRMACT_RETx,HHFAMINC_IMP_HIx,R_RACE_WHITEx,HHSIZE1&2x,URBANx,HHSIZE1&2&3x
0,9000013002,1,1,1,900001300201,1,2,1,1,1,2,75,2,3,1,50,15,1435,1450,1,1,3,2,1,2,2,1,-1,-1,-1,-1,-1,-1,0,2,1,1,2,1,-1.0,-1,-1,15,2,3.902424,1.608362e+06,0.0,1.569852e+06,90000130020101,3.902424,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,39,2,2,1,1,2,1,7,3,2,3,11,0,0,1,False,0,False,1,1,False,True,False
1,9000013002,1,2,2,900001300201,1,2,1,1,1,2,-9,2,3,15,1,10,1605,1615,1,1,3,2,1,2,2,1,-1,-1,-1,-1,-1,-1,0,2,1,1,2,1,-1.0,-1,-1,1,1,3.902424,1.608362e+06,0.0,1.569852e+06,90000130020102,3.902424,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,39,2,2,1,1,2,1,7,3,2,3,11,0,0,1,False,0,False,1,1,False,True,False
2,9000013002,2,1,1,900001300202,1,-1,1,1,1,2,10,2,2,1,80,30,700,730,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,12,-1,17.077067,2.208973e+06,0.0,2.274239e+06,90000130020201,17.077067,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,0,0,1,False,1,False,1,1,False,True,False
3,9000013002,2,2,2,900001300202,1,-1,1,1,1,2,10,2,5,12,80,10,740,750,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,12,-9,4.748291,2.208973e+06,0.0,2.274239e+06,90000130020202,4.748291,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,0,0,1,False,1,False,1,1,False,True,False
4,9000013002,2,3,3,900001300202,1,-1,1,1,1,2,-9,2,2,12,1,30,800,830,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,-9,14.098819,2.208973e+06,0.0,2.274239e+06,90000130020203,14.098819,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,0,0,1,False,1,False,1,1,False,True,False


In [9]:
df.describe()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,PUBTRANSy,HHFAMINC_IMP_LOWx,EDUC_HIx,R_SEX_Mx,HHFAMINC_IMP_HIx,R_RACE_WHITEx
count,3.107400e+04,31074.000000,31074.000000,31074.000000,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.00000,31074.000000,31074.000000,3.107400e+04,3.107400e+04,3.107400e+04,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000
mean,9.000116e+09,1.681438,2.437826,2.435702,7.619426e+11,1.048529,1.141308,0.775407,1.722276,3.810066,1.988704,95.176611,1.989026,3.156723,7.185943,26.856311,24.547821,1329.818337,1363.067613,1.041063,1.076945,3.906159,1.997265,1.243644,1.129336,0.185267,-0.226427,-0.644236,-0.880543,-0.954464,-0.976830,-0.993338,-0.999710,0.414140,1.583124,3.333526,0.986774,1.423505,3.786606,-0.802894,-0.986677,82.358853,7.43390,-0.210272,13.965150,8.166137e+06,5.815804e+06,2.302368e+06,9.000116e+13,7.527313,397.951262,5.351805,2.058506,1.654921,1.775729,5.143046,2.652346,1.924117,1.993242,53.722565,7.260636,2.677029,2.821941,2.134453,5.812737,2.292109,3.530926,1.653086,4.137478,1.188968,202211.691543,1.303694,1029.494240,46.800830,1.430617,1.126504,0.855892,2.859014,1.916071,1.268198,4.716097,4.710755,1.503926,1.724657,7.415814,0.010974,0.201937,0.503122,0.496074,0.478085,0.868636
std,5.829379e+04,0.999429,1.791504,1.789758,3.243517e+11,0.214885,1.341822,0.787571,0.447884,1.966057,0.105681,164.311604,0.104181,1.445374,6.851472,27.561324,46.483373,422.927312,425.447364,0.514696,1.094555,5.054558,3.477865,0.429287,0.982799,1.336076,1.215187,0.911104,0.563497,0.351382,0.254039,0.137048,0.029476,3.387813,0.891605,14.903745,0.833239,0.975625,15.956259,8.739298,0.207546,909.149169,6.99431,4.073497,85.416700,1.083988e+07,9.561997e+06,5.831630e+06,5.829379e+08,32.176230,68.455342,4.189875,0.761605,0.813206,0.417108,2.485935,1.011683,0.264816,0.794912,24.895858,2.861425,11.855930,1.446787,1.078400,3.251390,0.856474,1.552468,1.166259,1.665612,0.391490,20.136521,0.947416,4.982031,20.765643,0.931312,0.872042,0.646882,12.512009,0.277285,0.443029,2.409187,19.318665,0.499993,1.567172,2.445626,0.104181,0.401452,0.499998,0.499993,0.499528,0.337803
min,9.000013e+09,1.000000,1.000000,1.000000,-1.000000e+00,1.000000,-1.000000,-9.000000,1.000000,1.000000,1.000000,-9.000000,1.000000,-9.000000,-9.000000,1.000000,-9.000000,-9.000000,-9.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.0

In [10]:
df.groupby(['R_SEX_Mx']).size()

R_SEX_Mx
0    15659
1    15415
dtype: int64

## Model 1.1: One Independent Variable

Let's start with variable representing traveler gender

In [11]:
endogVar = df['PUBTRANSy'] # Set up dependent variable, should be only one column (array with one dimention)
exogVar1 = df['R_SEX_Mx'] # Set up independent variables with only one variable

logit_mod = sm.Logit(endogVar, exogVar1).fit() # Solve the model and save results in object logit_mod
print("Type of the created object is: ", type(logit_mod))
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Parameters: ", logit_mod.params)
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.380259
         Iterations 8
Type of the created object is:  <class 'statsmodels.discrete.discrete_model.BinaryResultsWrapper'>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Parameters:  R_SEX_Mx   -4.461129
dtype: float64
                           Logit Regression Results                           
Dep. Variable:              PUBTRANSy   No. Observations:                31074
Model:                          Logit   Df Residuals:                    31073
Method:                           MLE   Df Model:                            0
Date:                Thu, 16 Oct 2025   Pseudo R-squ.:                  -5.293
Time:                        16:11:25   Log-Likelihood:                -11816.
converged:                       True   LL-Null:                       -1877.8
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025  

### Model 1.1 discussion

Based on the output, logistic regression model shows a **statistically significant, strong negative relationship** between being *male* and the likelihood of *using public transit*.

However, the model has several major <span style = "color:red;">warning signs</span> in the model results.

    1. Coefficient (-4.4611) is the log-odds of the outcome. A negative coefficient means that when R_SEX_Mx is 1 (i.e., the person is male), the log-odds of using public transit decrease. In other words: being male is strongly associated with a lower likelihood of using public transit compared to not being male.
    2. P-value (P>|z| = 0.000) is extremely low (much less than 0.05). This means the relationship is statistically significant and not due to random chance. We can confidently reject the null hypothesis that there is no relationship between gender and public transit use.

    3. Calculating the impact of the odds ratio. To make the coefficient easier to understand, we convert it from log-odds to an odds ratio:

$
\text{Odds Ratio} = e^{\beta_1} = e^{−4.4611}≈0.0115
$

*Conclusion:* The odds of a male using public transit are 1.15% of the odds of a non-male using it. 

    4. What are signes of flawed model?

    - Pseudo R-squared *(-5.293)* is a major error. Pseudo R-squared is like R-squared in linear regression; it should be between 0 and 1. A negative value means your model is worse at predicting the outcome than a model with no variables at all.

    - Df Model ("Degrees of Freedom for the Model") is *0*. This almost always indicates that we have forgotten to add an intercept to your model. Our model is just *y ~ x* when it should be *y ~ constant + x*.  

    - LLR p-value *(nan)* (Likelihood-Ratio test) compares our model to the null (intercept-only) model. Since model is worse than the null model (as shown by the Pseudo R-squared), this test fails, resulting in *nan*.

        5. Final Discussion and Recommendation

    - analysis has correctly identified a significant negative association between being male and using public transit.

    - to improve the model we need to ensure an intercept term is included. 


## Model 1.2 with **Intercept**

In [23]:
### Add a constant (intercept) to the independent variable(s)
exogVar2_const = sm.add_constant(exogVar1)
logit_mod2 = sm.Logit(endogVar, exogVar2_const).fit() # Solve the model and save results in object logit_mod
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Parameters: ", logit_mod2.params)
print(logit_mod2.summary())

Optimization terminated successfully.
         Current function value: 0.060421
         Iterations 9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Parameters:  const      -4.542263
R_SEX_Mx    0.081133
dtype: float64
                           Logit Regression Results                           
Dep. Variable:              PUBTRANSy   No. Observations:                31074
Model:                          Logit   Df Residuals:                    31072
Method:                           MLE   Df Model:                            1
Date:                Thu, 16 Oct 2025   Pseudo R-squ.:               0.0001477
Time:                        16:20:15   Log-Likelihood:                -1877.5
converged:                       True   LL-Null:                       -1877.8
Covariance Type:            nonrobust   LLR p-value:                    0.4564
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------

### Model 1.2 Discussion

Based on the output, logistic regression model shows that there is no statistically significant relationship between being *male* and the likelihood of *using public transit*.

This model is now statistically valid, and the previous <span style = "color:red;">warning signs</span> have been resolved.

    1. Coefficient (0.0811): The coefficient for R_SEX_Mx is the log-odds of the outcome. A positive coefficient suggests that when R_SEX_Mx is 1 (i.e., the person is male), the log-odds of using public transit increase slightly. However, as we will see next, this effect is not statistically meaningful.

    2. P-value (P>|z| = 0.457): This value is very high, much larger than the standard significance level of 0.05. This means we fail to reject the null hypothesis. In plain English, there is no statistical evidence to conclude that gender has a relationship with public transit use in this model. The small positive effect observed is likely due to random chance.

    3. Calculating the Impact (Odds Ratio): To make the (non-significant) coefficient easier to understand, we can convert it to an odds ratio:

$
\text{Odds Ratio} = e^{\beta_1} = e^{0.0811}≈1.084
$

*Conclusion:* The odds of a male using public transit are 1.084 times the odds of a non-male using it (or 8.4% higher). Crucially, because this result is not statistically significant, this difference should be considered noise in the data and not a real effect.

    4. Model parameters

    - Pseudo R-squared (0.0001477): This value is now positive and between 0 and 1, as it should be. However, it is extremely close to zero, indicating that gender (R_SEX_Mx) explains a negligible, almost zero, amount of the variation in public transit use.

    - Df Model (1): The "Degrees of Freedom for the Model" is now 1, which is correct for a model with one predictor and an intercept.  

    - LLR p-value (0.4564): This is now a valid number. Since it is high, it confirms that your model is not a statistically significant improvement over a basic model with no variables.

        5. Final Discussion and Recommendation

After correctly specifying the model by including an intercept, the analysis shows that there is **no statistically significant evidence** of a relationship between *gender* and the *use of public transit*.

The initial strong negative relationship found in the previous model was likely an artifact of the incorrect model specification. This corrected model, while statistically valid, demonstrates that gender is not a meaningful predictor of public transit use on its own. To build a better model, we should investigate other variables that might influence this choice.

## Model 1.3 with Intercept to Explore Impacts of Ture/False Colunms and Discuss Results in the Class

In [19]:
exogVar2 = df['PRMACT_RETx'] # Set up a new independent variables with only one variable
exogVar3 = df['PRMACT_RETx'].astype(int) # Uncomment this line after discussion the error


### Add a constant (intercept) to the independent variable(s)
exogVar3_const = sm.add_constant(exogVar3)

logit_mod3 = sm.Logit(endogVar, exogVar3_const).fit() # Solve the model and save results in object logit_mod
print("Type of the created object is: ", type(logit_mod3))
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Parameters: ", logit_mod3.params)
print(logit_mod3.summary())

Optimization terminated successfully.
         Current function value: 0.060012
         Iterations 9
Type of the created object is:  <class 'statsmodels.discrete.discrete_model.BinaryResultsWrapper'>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Parameters:  const         -4.373991
PRMACT_RETx   -0.796343
dtype: float64
                           Logit Regression Results                           
Dep. Variable:              PUBTRANSy   No. Observations:                31074
Model:                          Logit   Df Residuals:                    31072
Method:                           MLE   Df Model:                            1
Date:                Thu, 16 Oct 2025   Pseudo R-squ.:                0.006921
Time:                        16:15:58   Log-Likelihood:                -1864.8
converged:                       True   LL-Null:                       -1877.8
Covariance Type:            nonrobust   LLR p-value:                 3.426e-07
                  coef    std err        

### Model 1.3 Results Discussion
    - LLR p-value **(3.426e-07)**: This p-value is very low, confirming that the model as a whole is a statistically **significant improvement** over an intercept-only model.
    - Pseudo R-squared (0.006921): This value is positive and between 0 and 1, as it should be. However, it is very low, indicating that retirement status by itself explains only about 0.7% of the variation in public transit use. It is a weak predictor.

## Model 1.4 with Another Variable to Discuss Results in Class

In [25]:
exogVar4 = df['HHFAMINC_IMP_HIx'] # Set up independent variables with only one variable

### Add a constant (intercept) to the independent variable(s)
exogVar4_const = sm.add_constant(exogVar4)

logit_mod4 = sm.Logit(endogVar, exogVar4_const).fit() # Solve the model and save results in object logit_mod
print("Type of the created object is: ", type(logit_mod4))
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Parameters: ", logit_mod4.params)
print(logit_mod4.summary())



Optimization terminated successfully.
         Current function value: 0.060402
         Iterations 9
Type of the created object is:  <class 'statsmodels.discrete.discrete_model.BinaryResultsWrapper'>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Parameters:  const              -4.435068
HHFAMINC_IMP_HIx   -0.143595
dtype: float64
                           Logit Regression Results                           
Dep. Variable:              PUBTRANSy   No. Observations:                31074
Model:                          Logit   Df Residuals:                    31072
Method:                           MLE   Df Model:                            1
Date:                Thu, 16 Oct 2025   Pseudo R-squ.:               0.0004590
Time:                        16:27:05   Log-Likelihood:                -1876.9
converged:                       True   LL-Null:                       -1877.8
Covariance Type:            nonrobust   LLR p-value:                    0.1892
                       coef    

## Model 2: Two Independent Variables


In [26]:
### Note the shape of each array!

print("Shape of the dependent variable array: ", endogVar.shape)

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

exogVar21 = df[['HHVEHCNT', "R_RACE_WHITEx"]]
print("Shape of the independent variable array without an intercept: ", exogVar21.shape)

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")

exogVar21_const = sm.add_constant(exogVar21, prepend = False, has_constant = 'skip')
print("Shape of the independent variable array with an intercept: ", exogVar21_const.shape)

Shape of the dependent variable array:  (31074,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Shape of the independent variable array without an intercept:  (31074, 2)
~~~~~~~~~~~~~~~~~~~~~~~~~~~
Shape of the independent variable array with an intercept:  (31074, 3)


### Some notes on add_constant

Two additional parameters that can be useful:

**prepend** (True or False) - A column-ordered design matrix. If *True*, the constant is in the first column. Else the constant is appended (last column). Default *True*

**has_constantstr** {‘raise’, ‘add’, ‘skip’} - Behavior if data already has a constant. The default will return data without adding another constant. If *raise*, will raise an error if any column has a constant value. Using *add* will add a column of 1s if a constant column is present.



In [27]:
### Let's use two variables to constract the next model

logit_mod21 = sm.Logit(endogVar, exogVar21).fit()
print("Parameters: ", logit_mod21.params)
print(logit_mod21.summary())

Optimization terminated successfully.
         Current function value: 0.049727
         Iterations 10
Parameters:  HHVEHCNT        -2.426260
R_RACE_WHITEx   -1.602586
dtype: float64
                           Logit Regression Results                           
Dep. Variable:              PUBTRANSy   No. Observations:                31074
Model:                          Logit   Df Residuals:                    31072
Method:                           MLE   Df Model:                            1
Date:                Thu, 16 Oct 2025   Pseudo R-squ.:                  0.1771
Time:                        16:30:10   Log-Likelihood:                -1545.2
converged:                       True   LL-Null:                       -1877.8
Covariance Type:            nonrobust   LLR p-value:                1.142e-146
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
HHVEHCNT         -2.4

In [28]:
dtripv.groupby("PUBTRANSy").size()

PUBTRANSy
0    30733
1      341
dtype: int64

## Model 3: Multiple Independent Variables + *from_formula*

In [29]:
#### Now four variables

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

logit_mod31 = sm.Logit.from_formula("PUBTRANSy ~ 1 + HHVEHCNT + HHFAMINC_IMP_LOWx + HHSIZE + URBANx", data = df)

results_logit_mod31 = logit_mod31.fit()
print("Parameters: ", results_logit_mod31.params)
print(results_logit_mod31.summary())

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Optimization terminated successfully.
         Current function value: 0.047471
         Iterations 10
Parameters:  Intercept           -2.574240
URBANx[T.True]       0.709475
HHVEHCNT            -2.046957
HHFAMINC_IMP_LOWx   -0.299278
HHSIZE               0.145048
dtype: float64
                           Logit Regression Results                           
Dep. Variable:              PUBTRANSy   No. Observations:                31074
Model:                          Logit   Df Residuals:                    31069
Method:                           MLE   Df Model:                            4
Date:                Thu, 16 Oct 2025   Pseudo R-squ.:                  0.2145
Time:                        16:33:14   Log-Likelihood:                -1475.1
converged:                       True   LL-Null:                       -1877.8
Covariance Type:            nonrobust   LLR p-value:                5.225e-173
                

In [30]:
### Another available tool useful for discussions is marginal effects calculation
margeff = results_logit_mod31.get_margeff()
print("Marginal effects: ")
print(margeff.summary())

Marginal effects: 
        Logit Marginal Effects       
Dep. Variable:              PUBTRANSy
Method:                          dydx
At:                           overall
                       dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
URBANx[T.True]        0.0072      0.003      2.846      0.004       0.002       0.012
HHVEHCNT             -0.0209      0.001    -16.700      0.000      -0.023      -0.018
HHFAMINC_IMP_LOWx    -0.0031      0.001     -2.413      0.016      -0.006      -0.001
HHSIZE                0.0015      0.000      3.709      0.000       0.001       0.002


In [32]:
### We also can calculate odds ratios using Numpy
### Get the coefficients (log-odds) from the model parameters
coefficients = results_logit_mod31.params
print("--- Model Coefficients (Log-Odds) ---")
print(coefficients)

### Calculate the Odds Ratios by exponentiating the coefficients
odds_ratios = np.exp(coefficients)
print("\n--- Calculated Odds Ratios ---")
print(odds_ratios)

--- Model Coefficients (Log-Odds) ---
Intercept           -2.574240
URBANx[T.True]       0.709475
HHVEHCNT            -2.046957
HHFAMINC_IMP_LOWx   -0.299278
HHSIZE               0.145048
dtype: float64

--- Calculated Odds Ratios ---
Intercept            0.076212
URBANx[T.True]       2.032923
HHVEHCNT             0.129127
HHFAMINC_IMP_LOWx    0.741353
HHSIZE               1.156095
dtype: float64


In [33]:
df.describe()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,PUBTRANSy,HHFAMINC_IMP_LOWx,EDUC_HIx,R_SEX_Mx,HHFAMINC_IMP_HIx,R_RACE_WHITEx
count,3.107400e+04,31074.000000,31074.000000,31074.000000,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.00000,31074.000000,31074.000000,3.107400e+04,3.107400e+04,3.107400e+04,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000
mean,9.000116e+09,1.681438,2.437826,2.435702,7.619426e+11,1.048529,1.141308,0.775407,1.722276,3.810066,1.988704,95.176611,1.989026,3.156723,7.185943,26.856311,24.547821,1329.818337,1363.067613,1.041063,1.076945,3.906159,1.997265,1.243644,1.129336,0.185267,-0.226427,-0.644236,-0.880543,-0.954464,-0.976830,-0.993338,-0.999710,0.414140,1.583124,3.333526,0.986774,1.423505,3.786606,-0.802894,-0.986677,82.358853,7.43390,-0.210272,13.965150,8.166137e+06,5.815804e+06,2.302368e+06,9.000116e+13,7.527313,397.951262,5.351805,2.058506,1.654921,1.775729,5.143046,2.652346,1.924117,1.993242,53.722565,7.260636,2.677029,2.821941,2.134453,5.812737,2.292109,3.530926,1.653086,4.137478,1.188968,202211.691543,1.303694,1029.494240,46.800830,1.430617,1.126504,0.855892,2.859014,1.916071,1.268198,4.716097,4.710755,1.503926,1.724657,7.415814,0.010974,0.201937,0.503122,0.496074,0.478085,0.868636
std,5.829379e+04,0.999429,1.791504,1.789758,3.243517e+11,0.214885,1.341822,0.787571,0.447884,1.966057,0.105681,164.311604,0.104181,1.445374,6.851472,27.561324,46.483373,422.927312,425.447364,0.514696,1.094555,5.054558,3.477865,0.429287,0.982799,1.336076,1.215187,0.911104,0.563497,0.351382,0.254039,0.137048,0.029476,3.387813,0.891605,14.903745,0.833239,0.975625,15.956259,8.739298,0.207546,909.149169,6.99431,4.073497,85.416700,1.083988e+07,9.561997e+06,5.831630e+06,5.829379e+08,32.176230,68.455342,4.189875,0.761605,0.813206,0.417108,2.485935,1.011683,0.264816,0.794912,24.895858,2.861425,11.855930,1.446787,1.078400,3.251390,0.856474,1.552468,1.166259,1.665612,0.391490,20.136521,0.947416,4.982031,20.765643,0.931312,0.872042,0.646882,12.512009,0.277285,0.443029,2.409187,19.318665,0.499993,1.567172,2.445626,0.104181,0.401452,0.499998,0.499993,0.499528,0.337803
min,9.000013e+09,1.000000,1.000000,1.000000,-1.000000e+00,1.000000,-1.000000,-9.000000,1.000000,1.000000,1.000000,-9.000000,1.000000,-9.000000,-9.000000,1.000000,-9.000000,-9.000000,-9.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.0

### Model 3 Discussion

Based on the output, model is **statistically valid**. It identifies several key factors that influence the likelihood of using public transit.

This model successfully explains about **21.5%** of the variation in public transit use **(Pseudo R-squ.: 0.2145)**, which is a reasonably good fit for this type of analysis. All included variables are statistically significant predictors.


1. Model Diagnostics

The model is statistically sound, with no warning signs.

- Df Model **(4)**: This correctly reflects the four predictor variables in the model.

- LLR p-value **(5.225e-173)**: The extremely low p-value confirms that the model as a whole is highly significant and far better than a model with no variables.

2. Analysis of Individual Predictors

**URBANx[T.True] (Living in an Urban Area)**

        - Coefficient (0.7095): The positive coefficient indicates that living in an urban area is associated with a higher likelihood of using public transit.

        - P-value (0.004): This relationship is statistically significant.

        - Odds Ratio (2.03): The odds of using public transit for someone living in an urban area are about 103% higher (or just over double) than for someone in a non-urban area, holding other factors constant.

        - Marginal Effect (dy/dx = 0.0072): Living in an urban area increases the probability of using public transit by 0.72 percentage points.


**HHVEHCNT (Household Vehicle Count)**

        - Coefficient (-2.0470): The large negative coefficient shows a very strong negative relationship between car ownership and transit use.

        - P-value (0.000): This is the most significant predictor in the model.

        - Odds Ratio (0.129): For each additional vehicle a household owns, the odds of using public transit decrease by about 87%. This is a powerful and expected finding.

        - Marginal Effect (dy/dx = -0.0209): For each additional vehicle a household owns, the probability of using public transit decreases by 2.09 percentage points.


**HHFAMINC_IMP_LOWx (Low Household Income)**

        - Coefficient (-0.2993): The negative coefficient suggests that being in a low-income household is associated with a lower likelihood of using public transit.

        - P-value (0.015): The relationship is statistically significant.

        - Odds Ratio (0.741): The odds of using public transit for a low-income household are 25.9% lower than for a non-low-income household, holding other factors constant. This is an interesting result that may warrant further investigation.

        - Marginal Effect (dy/dx = -0.0031): Being in a low-income household decreases the probability of using public transit by 0.31 percentage points.

**HHSIZE (Household Size)**

        - Coefficient (0.1450): The positive coefficient indicates that larger households are more likely to use public transit.

        - P-value (0.000): The relationship is highly significant.

        - Odds Ratio (1.156): For each additional person in a household, the odds of using public transit increase by 15.6%.
   
        - Marginal Effect (dy/dx = 0.0015): For each additional person in a household, the probability of using public transit increases by 0.15 percentage points.


3. Final Discussion and Recommendation

This is a strong and interpretable model. It confirms that the number of vehicles is the dominant factor discouraging public transit use, while living in an urban area is the strongest factor encouraging it. Household size also plays a significant positive role.

The finding that low-income households are less likely to use transit is noteworthy and could be a valuable insight for policy or further research. 

This analysis of the marginal effects provides a direct and intuitive interpretation of your logistic regression model. Unlike coefficients (which are in log-odds), marginal effects show how the probability of using public transit changes for a one-unit increase in each predictor, holding all other variables constant.

The marginal effects confirm the findings from the odds ratios but state them in more direct, probabilistic terms. The most impactful factor is vehicle ownership; each additional car reduces the likelihood of using public transit by a substantial 2.09 percentage points.


### Odds Ratio vs. Marginal Effects

<span style = "color:green;">An odds ratio tells you how the odds of an outcome change for a one-unit increase in a predictor. It's a multiplicative and relative measure.</span>

<span style = "color:purple;">A marginal effect tells you how the probability of an outcome changes for a one-unit increase in a predictor. It's an additive and absolute measure.</span>

|Feature	|Odds Ratio	|Marginal Effect|
|-------------------|------------------------|-------------------------------|
|Interpretation	|Multiplicative change in odds	|Additive change in probability|
|Scale	|Relative	|Absolute (percentage points)|
|Constancy	|Constant across all data points	|Varies; usually reported as an average (AME)|
|Best For...	|Technical reporting, measuring association	|Explaining practical impact and magnitude|
|Primary Audience	|Researchers, statisticians	|General audience, business, policy|



<div class="alert alert-block alert-info">
<b>Important:</b> It is important to remember that a coefficient in a logistic regression are "conditional" on the other variables being held fixed.
</div>

## Binary Probit Model

In [39]:
### Let's try Probit model for our next binary models. 
# + HHFAMINC_IMP_LOWx
probit_mod = sm.Probit.from_formula("PUBTRANSy ~ 1 + HHVEHCNT  + HHSIZE + URBANx", data = df).fit()
print("Parameters: ", probit_mod.params)
print(probit_mod.summary())


Optimization terminated successfully.
         Current function value: 0.048726
         Iterations 9
Parameters:  Intercept        -1.583439
URBANx[T.True]    0.235058
HHVEHCNT         -0.735130
HHSIZE            0.068119
dtype: float64
                          Probit Regression Results                           
Dep. Variable:              PUBTRANSy   No. Observations:                31074
Model:                         Probit   Df Residuals:                    31070
Method:                           MLE   Df Model:                            3
Date:                Thu, 16 Oct 2025   Pseudo R-squ.:                  0.1937
Time:                        16:46:25   Log-Likelihood:                -1514.1
converged:                       True   LL-Null:                       -1877.8
Covariance Type:            nonrobust   LLR p-value:                2.428e-157
                     coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------

### Probit Model Discussion

Overall, the model is statistically significant and explains about **19.5%** of the variation in public transit use (Pseudo R-squ.: 0.1947). However, the **quasi-separation warning** indicates a potential issue with the reliability of the coefficient estimates.

1. Analysis of Individual Predictors

The coefficients in a Probit model represent the change in the z-score of the outcome for a one-unit change in the predictor. A positive coefficient increases the probability, while a negative one decreases it.

**URBANx[T.True] (Living in an Urban Area)**

    - Coefficient (0.2244): The positive sign indicates that living in an urban area is associated with a higher probability of using public transit.

    - P-value (0.010): This effect is statistically significant.

**HHVEHCNT (Household Vehicle Count)**

    - Coefficient (-0.7559): The large negative coefficient shows a strong negative relationship. Each additional vehicle in a household is associated with a substantially lower probability of using public transit.

    - P-value (0.000): This is the most statistically significant predictor in the model.

**HHFAMINC_IMP_LOWx (Low Household Income)**

    - Coefficient (-0.1021): The negative sign suggests that being in a low-income household is associated with a lower probability of using public transit.

    - P-value (0.055): This is **not** statistically significant at the standard 0.05 level. We cannot confidently conclude that low income has a real effect on transit use in this model. This is the major difference from Logit Model.

**HHSIZE (Household Size)**

    - Coefficient (0.0703): The positive coefficient indicates that as household size increases, the probability of using public transit also increases.

    - P-value (0.000): This relationship is highly significant.

2. Signs of a Issues with the Model?

The model is generally well-specified, but there is one major <span style="color:red;">warning sign</span>:

- **Complete Quasi-Separation Warning:** It states that "A fraction 0.10 of observations can be perfectly predicted. "This mians that for 10% of  data, one or more predictors perfectly separate the outcome. For example, it might be that every household with 5 or more vehicles (HHVEHCNT >= 5) never uses public transit (all PUBTRANSy = 0).

- **Implication:** This can lead to unreliable and inflated coefficient estimates. While the directions of the relationships are likely correct, the specific magnitudes of the coefficients (and thus their practical impact) might be unstable.

3. Final Discussion and Recommendation

Analysis identifies a clear and statistically significant picture: public transit use is positively associated with living in an urban area and having a larger household, while it is strongly and negatively associated with owning vehicles.

However, the quasi-separation warning is a significant issue. It suggests that the model may be overfitting to a subset of the data, making the specific coefficient values less trustworthy. Before drawing firm conclusions about the size of each effect, it would be wise to investigate the source of the separation. It can be done by exploring the data to see which variable (likely HHVEHCNT) is perfectly predicting the outcome for a subset of your observations.


# Homework Questions

Create a binary models that identifies factor impacting choice to use sustainable (public transit + active) modes for trips with at least three variables that are statistically significant. 

Use 'TRPTRANS' to create a binary dependent variable SUSTAINABLE.



Sustainable modes to consider (= 1) 

- 08 = Public or commuter bus

- 09 = School bus

- 10 = Street car or trolley car

- 11 = Subway or elevated rail

- 12 = commuter rail

- 18 = Bicycle (including bikeshare, ebike, etc.) 

- 19 = E-scooter 

- 20 = Walked


Exclude from the analysis:

- 13 = Amtrack

- 14 = Airplane

- 21 = Other



In [35]:
dtripv.groupby("TRPTRANS").size()

TRPTRANS
1     11777
2      1881
3     10384
4      3302
6        10
7        55
8       190
9       650
10        9
11      109
12       33
13        4
14       71
15        9
16      109
17        9
18      298
19        6
20     2103
21       65
dtype: int64

In [40]:
### Define the lists for sustainable and excludeed modes
sustainable_modes = [8, 9, 10, 11, 12, 18, 19, 20]
excluded_modes = [13, 14, 21]

### Create a new DataFrame that excludes the specified modes
###    The '~' symbol means 'not'
dx = dtripv[~dtripv['TRPTRANS'].isin(excluded_modes)].copy()


### Create the new binary 'SUSTAINABLE' column in the filtered DataFrame
###    .isin() returns True/False, and .astype(int) converts True to 1 and False to 0
dx['SUSTAINABLE'] = dx['TRPTRANS'].isin(sustainable_modes).astype(int)



In [41]:
dx.groupby("SUSTAINABLE").size()


SUSTAINABLE
0    27536
1     3398
dtype: int64

In [ ]:
#### Create instrumental variables if needed

xxxxx



In [ ]:
#### Remove invalid entries, add all columns that you are planing to use, including those you have created at previous step
dmodel = dx.dropna(subset = ['SUSTAINABLE', xxxxxxxxxxxxxxxxx], inplace = False)


In [ ]:
#### Create and report results of an the model with at least 3 parameters

xxxxxx


__Discussion Question 1.__ Discuss Model Fit Results





xxxxxxxxxx

__Discussion Question 2.__ Discuss Individual Predictors, add discussion either on marginal effects or odds ratios




xxxxxxx

__Discussion Question 3.__ Provide  Conclusing Discussions and Recomendations, if any




xxxxx

# To submit your homework:

    1 - Run all code cells 

    2 - Answer all questions

    3 - Download as pdf (you can use print the page and save it as pdf)  

    4 - Inspect the pdf file (all cells are executed and all questions are answered?)

    5 - If it looks good

            - Rename it as following *WA7_first_last* replacing *first_last* with your first and last names
    
            - Upload pdf to UBLearns/BrightSpace
    
        else
    
            - Fix the issue
    
            - Repeat from step 3
    

<div class="alert alert-block alert-info">
<b>Tip:</b> you may need to install additional libriaries to enable download into pdf funciton using pip and command window, specifically: >pyppeteer-install and/or nbconvert[webpdf].
</div>